## Create training/testing model

In [ ]:
def modelling():
    connection_2018 = sqlite3.connect('/NBA_db_2018_19.db') # Define own folder
    data_model_df = pd.read_sql_query("SELECT * FROM modelling_data", connection_2018).iloc[:,2:]
    
    # Define type of estimator used in modelling
    estimators = ['RandomForestRegressor']
    types = ['train', 'test']
    coefficients = ['rmse', 'r2']
    
    # Create an empty table (estimators vs types)
    rmse_names = [x + '_' + y for y in types for x in estimators] 
    df_rmse = pd.DataFrame([[0.0] * 2 for j in range(len(rmse_names))], index = rmse_names, columns = coefficients)

    # Data preparation to put into scikit. Depended variable y and Independent variables, attributes, X
    y = data_model_df['DF_PTS']    
    X = data_model_df[learning_attributes].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(X)
    X = pd.DataFrame(x_scaled, columns=learning_attributes)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)    
    print ('Data shapes. X_train: {}, y_train: {}, X_test: {}, y_test: {}'.format(
        X_train.shape, y_train.shape, X_test.shape, y_test.shape))
    
    # Training the model. Currently only with Rand. Forest, but left flexibility for additional types
    for i in range(len(estimators)):
        est = estimators[i]
        if(est == 'RandomForestRegressor'):
            n_estimators = [100]
            param_grid = {'n_estimators': n_estimators}
            rf = GridSearchCV(RandomForestRegressor(max_depth=7), param_grid, cv=5)
            rf.fit(X_train, y_train) 
        
    #Calculating statistical significance
    train_rmse = np.sqrt(np.mean((y_train - rf.predict(X_train))**2.0 ))
    test_rmse = np.sqrt(np.mean((y_test - rf.predict(X_test))**2.0 ))
    train_r2  = r2_score(y_train, rf.predict(X_train))
    test_r2 = r2_score(y_test, rf.predict(X_test))

    # Adding above calculated measures into the created empty table
    for val in types:
        for coef in coefficients:
            df_rmse.loc[estimators[i] + '_' + val, coef] = eval(val + '_' + coef)  
    print ('Model performance: {}'.format(df_rmse))   
        
    # Save the trained model
    Nba_model_pickle = '' # Define own folder
    pickle.dump(rf, open(Nba_model_pickle, 'wb'))